<h1><center> Aggregated Data Noise Creation <br><br> 
<center> Scikit Mobility and DB Noise<br><br> 
Wendy Shi</center></center>

# Preparation

In [2]:
#Preparation
import pandas as pd
import numpy as np

## Load Count Data

In [3]:
ori_mt = pd.read_csv("WA_Data/Aggregate/WA_ori_indi.csv")
ori_mt = ori_mt.drop(columns = 'Unnamed: 0')
ori_mt.head(3)

,53001,53003,53005,53007,53009,53011,53015,53017,53021,53025,...,53057,53059,53061,53063,53065,53067,53071,53073,53075,53077
0,0.394639,0.000000,0.055023,0.030399,0.000545,0.024406,0.007736,0.015036,0.042820,0.023317,...,0.000000,0.000000,0.029200,0.085422,0.003922,0.004249,0.008499,0.046306,0.004140,0.026694
1,0.000000,0.917968,0.003904,0.000220,0.000000,0.001100,0.000275,0.000110,0.003299,0.000385,...,0.002694,0.000000,0.003739,0.013800,0.002804,0.001869,0.001649,0.000110,0.016439,0.000330
2,0.001535,0.000008,0.738961,0.000639,0.000240,0.005779,0.000496,0.000184,0.161487,0.004964,...,0.000192,0.000016,0.005532,0.010416,0.000919,0.002710,0.006739,0.002382,0.000576,0.023861


In [4]:
column_names = ['S', 'E', 'I', 'R']
pop_2020 = pd.read_csv("../../Data/us_shapefile_county.csv")
pop_2020 = pop_2020[['GEO_ID','pop2020']]
pop_2020["GEO_ID"] = pop_2020["GEO_ID"].apply(lambda x: f"{int(x):05d}")

rename = {'GEO_ID':'GEOID'}
pop_2020 = pop_2020.rename(columns = rename)

print(pop_2020.shape)
pop_2020.tail(3)

(3109, 2)


,GEOID,pop2020
3106,06037,9989165.0
3107,46113,14309.0
3108,51515,6599.0


In [5]:
county_list = ori_mt.columns.tolist()
county_pop = pd.DataFrame()
county_pop['GEOID'] = county_list
county_pop.head(3)

,GEOID
0,53001
1,53003
2,53005


In [6]:
# Get list for the population
joined = pd.merge(county_pop, pop_2020, on = 'GEOID', how = 'left')
pop_list = joined['pop2020'].tolist()
print(f'Length of the population list: {len(pop_list)}')
joined.head()

Length of the population list: 34


,GEOID,pop2020
0,53001,20605.0
1,53003,22290.0
2,53005,207230.0
3,53007,79185.0
4,53009,77280.0


In [7]:
pop_list = joined['pop2020'].tolist()
pop_list[:5]

[20605.0, 22290.0, 207230.0, 79185.0, 77280.0]

In [8]:
ori_count = ori_mt.multiply(pop_list, axis=0)
ori_count.head(3)

,53001,53003,53005,53007,53009,53011,53015,53017,53021,53025,...,53057,53059,53061,53063,53065,53067,53071,53073,53075,53077
0,8131.543909,0.000000,1133.746459,626.366856,11.225212,502.889518,159.398017,309.815864,882.301700,480.439093,...,0.000000,0.00000,601.671388,1760.113314,80.821530,87.556657,175.113314,954.143059,85.311615,550.035411
1,0.000000,20461.504289,87.012866,4.902133,0.000000,24.510666,6.127667,2.451067,73.531999,8.578733,...,60.051133,0.00000,83.336266,307.608863,62.502199,41.668133,36.766000,2.451067,366.434462,7.353200
2,318.050839,1.656515,153134.853078,132.521183,49.695444,1197.660192,102.703917,38.099840,33464.911751,1028.695683,...,39.756355,3.31303,1146.308233,2158.438769,190.499201,561.558513,1396.441966,493.641407,119.269065,4944.696643


# Differential Pricavy Noise

In [9]:
#from opendp.trans import make_base_laplace
import opendp.prelude as dp
dp.enable_features("contrib")

In [41]:
count_noise = ori_count.astype(int).copy()
DP_df = pd.DataFrame()

for i in range(34):
    
    if i % 10 == 0: 
        print(i)
        
    epsilon = 0.001 #Privacy budget
    sensitivity = pop_list[i] * 0.0001 # Define sensitivity for each county
    scale = sensitivity / epsilon
    
    # Create the Laplace mechanism
    laplace_mechanism = dp.m.make_laplace(
        dp.atom_domain(T=int), dp.absolute_distance(T=int), scale
    )
    
    def apply_dp_noise(count):
        return laplace_mechanism(count)
    
    new_df_row = count_noise.iloc[[i]].applymap(lambda x: max(laplace_mechanism(x), 0))
    DP_df = pd.concat([DP_df, new_df_row], axis = 0)

0
10
20
30


In [42]:
#Original
ori_count.head(3)

,53001,53003,53005,53007,53009,53011,53015,53017,53021,53025,...,53057,53059,53061,53063,53065,53067,53071,53073,53075,53077
0,8131.543909,0.000000,1133.746459,626.366856,11.225212,502.889518,159.398017,309.815864,882.301700,480.439093,...,0.000000,0.00000,601.671388,1760.113314,80.821530,87.556657,175.113314,954.143059,85.311615,550.035411
1,0.000000,20461.504289,87.012866,4.902133,0.000000,24.510666,6.127667,2.451067,73.531999,8.578733,...,60.051133,0.00000,83.336266,307.608863,62.502199,41.668133,36.766000,2.451067,366.434462,7.353200
2,318.050839,1.656515,153134.853078,132.521183,49.695444,1197.660192,102.703917,38.099840,33464.911751,1028.695683,...,39.756355,3.31303,1146.308233,2158.438769,190.499201,561.558513,1396.441966,493.641407,119.269065,4944.696643


In [43]:
DP_df.head(3)

,53001,53003,53005,53007,53009,53011,53015,53017,53021,53025,...,53057,53059,53061,53063,53065,53067,53071,53073,53075,53077
0,8769,2917,1727,2573,0,2533,403,1729,941,0,...,283,162,0,10186,739,0,6177,1911,2834,0
1,3807,19471,176,0,0,1451,4340,0,0,0,...,140,2777,0,0,0,7006,0,1477,5251,0
2,17329,1062,151470,18421,7373,47084,4987,9675,5730,647,...,0,0,0,0,10118,49019,0,0,0,0


In [44]:
#Turn count to weight
divisor = [DP_df.iloc[i].sum() for i in range(34)]
noise_mt = DP_df.div(divisor, axis=0)
noise_mt.head(3)

,53001,53003,53005,53007,53009,53011,53015,53017,53021,53025,...,53057,53059,53061,53063,53065,53067,53071,53073,53075,53077
0,0.167060,0.055572,0.032902,0.049019,0.00000,0.048257,0.007678,0.03294,0.017927,0.000000,...,0.005392,0.003086,0.0,0.194056,0.014079,0.000000,0.11768,0.036407,0.053991,0.0
1,0.050658,0.259092,0.002342,0.000000,0.00000,0.019308,0.057750,0.00000,0.000000,0.000000,...,0.001863,0.036952,0.0,0.000000,0.000000,0.093226,0.00000,0.019654,0.069873,0.0
2,0.041554,0.002547,0.363221,0.044173,0.01768,0.112906,0.011959,0.02320,0.013740,0.001551,...,0.000000,0.000000,0.0,0.000000,0.024263,0.117546,0.00000,0.000000,0.000000,0.0


In [45]:
# Calculations
import statistics

#noise_mt_np = ori_mt.to_numpy()
noise_mt_np = noise_mt.to_numpy()
Pii_skmob = [noise_mt_np[i][i] for i in range(34)]


data = Pii_skmob 
mean_value = sum(data) / len(data)  # Mean
median_value = statistics.median(data)  # Median
min_value = min(data)  # Min
max_value = max(data)  # Max

# Output
print(f"Mean: {mean_value}")
print(f"Median: {median_value}")
print(f"Min: {min_value}")
print(f"Max: {max_value}")

Mean: 0.23613477784349435
Median: 0.22819916569122028
Min: 0.04286659204233235
Max: 0.47278745538301964


In [46]:
noise_mt.to_csv('WA_P001_agg.csv')

## Original Matrix Summary Statistics

- Mean: 0.5674103700330021
- Median: 0.5791140792056435
- Min: 0.15297047314123088
- Max: 0.9704900914807166